In [2]:
import shapely
import shapely.vectorized
import pandas as pd
import geopandas

In [3]:
gdl = geopandas.read_file("data/GDL Shapefiles V5 11-21.shp")
gdl.head()

,gdlcode,continent,iso_code,geometry
0,AFGr101,Asia/Pacific,AFG,"POLYGON ((69.41367 33.91973, 69.40897 33.91796..."
1,AFGr102,Asia/Pacific,AFG,"POLYGON ((65.32353 33.12133, 65.34240 33.17651..."
2,AFGr103,Asia/Pacific,AFG,"POLYGON ((70.46561 33.92853, 70.45656 33.92963..."
3,AFGr104,Asia/Pacific,AFG,"POLYGON ((66.38873 34.93133, 66.36456 34.93081..."
4,AFGr105,Asia/Pacific,AFG,"POLYGON ((67.35538 34.88549, 67.35532 34.88847..."


In [4]:
grid_df = pd.read_csv('data/2_merged_da_data.csv')
grid_df.head()

,LAT,LON,LAT_25,LON_25,LAT_5,LON_5,knutson_precip,updated_precip,knutson_temp,updated_temp,...,precip_da_cat,temp_da_cat,temp_da_binary,precip_da_binary,n_da_impacts,max_da_cat,study_addition,total_study_prop,area,is_land
0,-88.75,-178.75,-88.75,-178.75,-87.5,-177.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.0,1685.654015,False
1,-88.75,-176.25,-88.75,-176.25,-87.5,-177.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.0,1685.654015,False
2,-88.75,-173.75,-88.75,-173.75,-87.5,-172.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.0,1685.654015,False
3,-88.75,-171.25,-88.75,-171.25,-87.5,-172.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.0,1685.654015,False
4,-88.75,-168.75,-88.75,-168.75,-87.5,-167.5,NaN,NaN,NaN,NaN,...,0.0,0.0,0,0,0,0.0,add,0.0,1685.654015,False


In [9]:
grid_df.columns

Index(['LAT', 'LON', 'LAT_25', 'LON_25', 'LAT_5', 'LON_5', 'knutson_precip',
       'updated_precip', 'knutson_temp', 'updated_temp', 'population',
       'diff_temp', 'diff_precip', 'index', 'all_study_prop',
       'all_study_prop_upper', 'all_study_prop_lower', 'WA', 'temp_study_prop',
       'temp_study_prop_upper', 'temp_study_prop_lower', 'precip_study_prop',
       'precip_study_prop_upper', 'precip_study_prop_lower', 'precip_da_cat',
       'temp_da_cat', 'temp_da_binary', 'precip_da_binary', 'n_da_impacts',
       'max_da_cat', 'study_addition', 'total_study_prop', 'area', 'is_land'],
      dtype='object')

In [6]:
import math
def area_cell(lat, lon, degrees): 
    # calculate the area of a gridcell given the center lat and lon and the size in degrees
    if lon <0:
        lon+=360
    R = 6371
    f0 = math.radians(lat-degrees*0.5)
    f1 = math.radians(lat+degrees*0.5)
    l0 = math.radians(lon-degrees*0.5)
    l1 = math.radians(lon+degrees*0.5)

    return (math.sin(f1)-math.sin(f0)) * (l1 - l0) * R**2

In [10]:
# create different grids
import numpy as np
degrees = 0.5
da_degrees = 2.5
lons = np.linspace(-180+degrees*.5,180-degrees*.5,int(360/degrees))
lats = np.linspace(-90+degrees*.5,90-degrees*.5,int(180/degrees))
xv, yv = np.meshgrid(lons,lats)

subgrid_df = pd.DataFrame({"lat_05": yv.ravel(),"lon_05": xv.ravel(),})
subgrid_df['LAT'] = subgrid_df['lat_05']//da_degrees*da_degrees+da_degrees*0.5
subgrid_df['LON'] = subgrid_df['lon_05']//da_degrees*da_degrees+da_degrees*0.5

rescale_degrees = 5
subgrid_df['LAT_5'] = subgrid_df['lat_05']//rescale_degrees*rescale_degrees+rescale_degrees*0.5
subgrid_df['LON_5'] = subgrid_df['lon_05']//rescale_degrees*rescale_degrees+rescale_degrees*0.5

rescale_degrees = 2.5
subgrid_df['LAT_25'] = subgrid_df['lat_05']//rescale_degrees*rescale_degrees+rescale_degrees*0.5
subgrid_df['LON_25'] = subgrid_df['lon_05']//rescale_degrees*rescale_degrees+rescale_degrees*0.5

subgrid_df['area'] = subgrid_df.apply(lambda x: area_cell(x['lat_05'], x['lon_05'], 0.5), axis=1)
print(subgrid_df.shape)
subgrid_df = subgrid_df.merge(grid_df,how="left")
print(subgrid_df.shape)
subgrid_df

(259200, 9)
(259200, 36)


,lat_05,lon_05,LAT,LON,LAT_5,LON_5,LAT_25,LON_25,area,knutson_precip,...,precip_study_prop_lower,precip_da_cat,temp_da_cat,temp_da_binary,precip_da_binary,n_da_impacts,max_da_cat,study_addition,total_study_prop,is_land
0,-89.75,-179.75,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-89.75,-179.25,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-89.75,-178.75,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-89.75,-178.25,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-89.75,-177.75,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259195,89.75,177.75,88.75,178.75,87.5,177.5,88.75,178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259196,89.75,178.25,88.75,178.75,87.5,177.5,88.75,178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259197,89.75,178.75,88.75,178.75,87.5,177.5,88.75,178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259198,89.75,179.25,88.75,178.75,87.5,177.5,88.75,178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
-180+2.5

-177.5

In [19]:
degrees=2.5
np.linspace(-180+degrees*.5,180-degrees*.5,int(360/degrees))

array([-178.75, -176.25, -173.75, -171.25, -168.75, -166.25, -163.75,
       -161.25, -158.75, -156.25, -153.75, -151.25, -148.75, -146.25,
       -143.75, -141.25, -138.75, -136.25, -133.75, -131.25, -128.75,
       -126.25, -123.75, -121.25, -118.75, -116.25, -113.75, -111.25,
       -108.75, -106.25, -103.75, -101.25,  -98.75,  -96.25,  -93.75,
        -91.25,  -88.75,  -86.25,  -83.75,  -81.25,  -78.75,  -76.25,
        -73.75,  -71.25,  -68.75,  -66.25,  -63.75,  -61.25,  -58.75,
        -56.25,  -53.75,  -51.25,  -48.75,  -46.25,  -43.75,  -41.25,
        -38.75,  -36.25,  -33.75,  -31.25,  -28.75,  -26.25,  -23.75,
        -21.25,  -18.75,  -16.25,  -13.75,  -11.25,   -8.75,   -6.25,
         -3.75,   -1.25,    1.25,    3.75,    6.25,    8.75,   11.25,
         13.75,   16.25,   18.75,   21.25,   23.75,   26.25,   28.75,
         31.25,   33.75,   36.25,   38.75,   41.25,   43.75,   46.25,
         48.75,   51.25,   53.75,   56.25,   58.75,   61.25,   63.75,
         66.25,   68

In [14]:
subgrid_df.LON_25.unique()

array([-178.75, -176.25, -173.75, -171.25, -168.75, -166.25, -163.75,
       -161.25, -158.75, -156.25, -153.75, -151.25, -148.75, -146.25,
       -143.75, -141.25, -138.75, -136.25, -133.75, -131.25, -128.75,
       -126.25, -123.75, -121.25, -118.75, -116.25, -113.75, -111.25,
       -108.75, -106.25, -103.75, -101.25,  -98.75,  -96.25,  -93.75,
        -91.25,  -88.75,  -86.25,  -83.75,  -81.25,  -78.75,  -76.25,
        -73.75,  -71.25,  -68.75,  -66.25,  -63.75,  -61.25,  -58.75,
        -56.25,  -53.75,  -51.25,  -48.75,  -46.25,  -43.75,  -41.25,
        -38.75,  -36.25,  -33.75,  -31.25,  -28.75,  -26.25,  -23.75,
        -21.25,  -18.75,  -16.25,  -13.75,  -11.25,   -8.75,   -6.25,
         -3.75,   -1.25,    1.25,    3.75,    6.25,    8.75,   11.25,
         13.75,   16.25,   18.75,   21.25,   23.75,   26.25,   28.75,
         31.25,   33.75,   36.25,   38.75,   41.25,   43.75,   46.25,
         48.75,   51.25,   53.75,   56.25,   58.75,   61.25,   63.75,
         66.25,   68

In [11]:
# join small grids with corresponding region code   
subgrid_df["GDLcode"] = np.nan
for i, row in gdl.iterrows():
    contains = shapely.vectorized.contains(row.geometry,xv,yv).ravel()
    idx = np.argwhere(contains==True).ravel()

    subgrid_df.loc[idx, "GDLcode"]=row['gdlcode']
    
    
subgrid_df.to_csv('data/subgrid.csv',index=False)

In [12]:
subgrid_df.head()

,lat_05,lon_05,LAT,LON,LAT_5,LON_5,LAT_25,LON_25,area,knutson_precip,...,precip_da_cat,temp_da_cat,temp_da_binary,precip_da_binary,n_da_impacts,max_da_cat,study_addition,total_study_prop,is_land,GDLcode
0,-89.75,-179.75,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-89.75,-179.25,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-89.75,-178.75,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-89.75,-178.25,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-89.75,-177.75,-88.75,-178.75,-87.5,-177.5,-88.75,-178.75,13.487286,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# percentage of area per region where change in temp, prec is attibutable to human intervention
total_area = subgrid_df.groupby("GDLcode")["area"].sum()

filter_prec= subgrid_df.loc[abs(subgrid_df.updated_precip).isin([2,3]) ]
filter_temp= subgrid_df.loc[abs(subgrid_df.updated_temp).isin([2,3]) ]

group_prec = filter_prec.groupby("GDLcode")["area"].sum()
group_temp = filter_temp.groupby("GDLcode")["area"].sum()

dat = {
    "updated_temp_perc":  group_temp/total_area,
    "updated_prec_perc":  group_prec/total_area,
    "total_area": total_area
}

pd.DataFrame(dat).to_excel("change_temp_prec_perc_of_area.xlsx")